# Introduction: Recurrent Neural Network Quickstart

The purpose of this notebook is to serve as a rapid introduction to recurrent neural networks. All of the details can be found in `Deep Dive into Recurrent Neural Networks` while this notebook focuses on using the pre-trained network.

In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
!conda install scikit-learn -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.6.7
  latest version: 4.6.8

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jwq/anaconda3/envs/py3

  added / updated specs:
    - scikit-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.1.23  |                0         126 KB
    certifi-2019.3.9           |           py36_0         155 KB
    scikit-learn-0.20.3        |   py36hd81dba3_0         5.7 MB
    ------------------------------------------------------------
                                           Total:         6.0 MB

The following NEW packages will be INSTALLED:

  scikit-learn       pkgs/main/linux-64::scikit-learn-0.20.3-py36hd81dba3_0

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-

In [7]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import HTML

InteractiveShell.ast_node_interactivity = 'all'

import warnings
warnings.filterwarnings('ignore', category = RuntimeWarning)
warnings.filterwarnings('ignore', category = UserWarning)

import pandas as pd
import numpy as np
from utils import get_data, generate_output, guess_human, seed_sequence, get_embeddings, find_closest

# Fetch Training Data

* Using patent abstracts from patent search for neural network
* 3000+ patents total


In [8]:
data = pd.read_csv('../data/neural_network_patent_query.csv')
data.head()

,patent_abstract,patent_date,patent_number,patent_title
0,""" A """"Barometer"""" Neuron enhances stability in...",1996-07-09,5535303,"""""""Barometer"""" neuron for a neural network"""
1,""" This invention is a novel high-speed neural ...",1993-10-19,5255349,"""Electronic neural network for solving """"trave..."
2,An optical information processor for use as a ...,1995-01-17,5383042,3 layer liquid crystal neural network with out...
3,A method and system for intelligent control of...,2001-01-02,6169981,3-brain architecture for an intelligent decisi...
4,A method and system for intelligent control of...,2003-06-17,6581048,3-brain architecture for an intelligent decisi...


In [9]:
training_dict, word_idx, idx_word, sequences = get_data('../data/neural_network_patent_query.csv', training_len = 50)

There are 16192 unique words.
There are 318563 sequences.


In [10]:
print(word_idx)

{'the': 1, 'a': 2, 'of': 3, '.': 4, ',': 5, 'and': 6, 'to': 7, 'network': 8, 'neural': 9, 'is': 10, 'for': 11, 'in': 12, 'The': 13, 'an': 14, 'data': 15, 'are': 16, 'by': 17, 'A': 18, 'input': 19, 'system': 20, 'with': 21, 'output': 22, 'or': 23, 'from': 24, 'which': 25, 'be': 26, 'as': 27, 'that': 28, 'on': 29, 'signal': 30, 'method': 31, 'each': 32, 'one': 33, 'image': 34, 'at': 35, 'using': 36, 'plurality': 37, 'first': 38, 'layer': 39, 'can': 40, 'includes': 41, 'set': 42, 'training': 43, 'control': 44, 'may': 45, 'processing': 46, 'based': 47, 'values': 48, 'second': 49, 'information': 50, 'signals': 51, 'model': 52, 'value': 53, 'used': 54, 'process': 55, 'learning': 56, 'neuron': 57, 'least': 58, 'pattern': 59, 'such': 60, 'device': 61, 'between': 62, 'time': 63, 'In': 64, 'circuit': 65, 'vector': 66, 'unit': 67, 'having': 68, 'into': 69, 'neurons': 70, 'apparatus': 71, 'function': 72, 'trained': 73, 'more': 74, 'invention': 75, 'An': 76, 'parameters': 77, 'provided': 78, 'corre

In [11]:
print(idx_word)

{1: 'the', 2: 'a', 3: 'of', 4: '.', 5: ',', 6: 'and', 7: 'to', 8: 'network', 9: 'neural', 10: 'is', 11: 'for', 12: 'in', 13: 'The', 14: 'an', 15: 'data', 16: 'are', 17: 'by', 18: 'A', 19: 'input', 20: 'system', 21: 'with', 22: 'output', 23: 'or', 24: 'from', 25: 'which', 26: 'be', 27: 'as', 28: 'that', 29: 'on', 30: 'signal', 31: 'method', 32: 'each', 33: 'one', 34: 'image', 35: 'at', 36: 'using', 37: 'plurality', 38: 'first', 39: 'layer', 40: 'can', 41: 'includes', 42: 'set', 43: 'training', 44: 'control', 45: 'may', 46: 'processing', 47: 'based', 48: 'values', 49: 'second', 50: 'information', 51: 'signals', 52: 'model', 53: 'value', 54: 'used', 55: 'process', 56: 'learning', 57: 'neuron', 58: 'least', 59: 'pattern', 60: 'such', 61: 'device', 62: 'between', 63: 'time', 64: 'In', 65: 'circuit', 66: 'vector', 67: 'unit', 68: 'having', 69: 'into', 70: 'neurons', 71: 'apparatus', 72: 'function', 73: 'trained', 74: 'more', 75: 'invention', 76: 'An', 77: 'parameters', 78: 'provided', 79: 'c

In [12]:
print(training_dict)

{'X_train': array([[  117,     7,   141, ...,     4,    13,   117],
       [    6,    41,     2, ...,    14,    22, 13944],
       [    2,    42,     3, ...,    24,  2109,   842],
       ...,
       [  212,     6,    14, ...,   461,    65,   433],
       [    3,   479,  2247, ...,    66,    10,   308],
       [ 5848,    12,   146, ...,  1517,    16, 15160]]), 'X_valid': array([[ 633, 5033,   21, ...,  182, 4039,    7],
       [  21,  609,  514, ...,    1,  622,    3],
       [   8,    5,   95, ...,    9,    8,   10],
       ...,
       [8925,    6,   74, ...,  124,    3,    1],
       [1654,  565,   47, ...,    3,    1, 1654],
       [   6,    2,   38, ...,  245,    9,    8]]), 'y_train': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8), 'y_valid': array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0

* Sequences of text are represented as integers
    * `word_idx` maps words to integers
    * `idx_word` maps integers to words
* Features are integer sequences of length 50
* Label is next word in sequence
* Labels are one-hot encoded

In [19]:
def print_idx(array):
    for i in range(len(array)):
        if (array[i] == 1):
            print("the lable of this one-hot vector is: {}".format(i))
training_dict['X_train'][:2]
training_dict['y_train'][:2]

for i in range(10):
    print_idx(training_dict['y_train'][i])
print("total label number: {}".format(len(training_dict['y_train'][0])))

array([[  117,     7,   141,   277,     4,    18,    81,   110,    10,
          219,    29,     1,   952,  2453,    19,     5,     6,     1,
          117,    10,   182,  2166,    21,     1,    81,   178,     4,
           13,   117,   894,    14,  6163,     7,   302,     1,     9,
            8,    29,    33,    23,    74,   428,     7,   692,     1,
           81,   183,     4,    13,   117],
       [    6,    41,     2,    87,     3,  1340,    79,     7,     1,
          409,   543,    22,   484,     6,     2,  2113,   728,    24,
            1,   178,     3,     1,  1820,    55,    14, 13942,  7240,
          244,     5,    14, 13943,  7240,   244,     5,     2,  2113,
         7240,   244,     5,     2,    38,  9292,   244,     2,    49,
         9292,   244,    14,    22, 13944]])

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8)

the lable of this one-hot vector is: 10
the lable of this one-hot vector is: 244
the lable of this one-hot vector is: 3
the lable of this one-hot vector is: 118
the lable of this one-hot vector is: 1127
the lable of this one-hot vector is: 1131
the lable of this one-hot vector is: 19
the lable of this one-hot vector is: 4
the lable of this one-hot vector is: 405
the lable of this one-hot vector is: 390
total label number: 16192


In [21]:
for i, sequence in enumerate(training_dict['X_train'][:10]):
    text = []
    for idx in sequence:
        text.append(idx_word[idx])
        
    print('Features: ' + ' '.join(text) + '\n')
    print('Label: ' + idx_word[np.argmax(training_dict['y_train'][i])] + '\n')
    

Features: user to provide samples . A recognition operation is performed on the user's handwritten input , and the user is not satisfied with the recognition result . The user selects an option to train the neural network on one or more characters to improve the recognition results . The user

Label: is

Features: and includes a number of amplifiers corresponding to the N bit output sum and a carry generation from the result of the adding process an augend input-synapse group , an addend input-synapse group , a carry input-synapse group , a first bias-synapse group a second bias-synapse group an output feedback-synapse

Label: group

Features: a set of physical parameters that fully define the smaller extraction problem . Next , models are created using machine learning techniques for all of the smaller simpler extraction problems . The machine learning is performed by first creating training data sets composed of the identified parameters from typical examples

Label: of

Features: no

# Make Recurrent Neural Network

* Embedding dimension = 100
* 64 LSTM cells in one layer
    * Dropout and recurrent dropout for regularization
* Fully connected layer with 64 units on top of LSTM
     * 'relu' activation
* Drop out for regularization
* Output layer produces prediction for each word
    * 'softmax' activation
* Adam optimizer with defaults
* Categorical cross entropy loss
* Monitor accuracy

In [23]:
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional
from keras.optimizers import Adam

from keras.utils import plot_model

In [24]:
model = Sequential()

# Embedding layer
model.add(
    Embedding(
        input_dim=len(word_idx) + 1,
        output_dim=100,
        weights=None,
        trainable=True))

# Recurrent layer
model.add(
    LSTM(
        64, return_sequences=False, dropout=0.1,
        recurrent_dropout=0.1))

# Fully connected layer
model.add(Dense(64, activation='relu'))

# Dropout for regularization
model.add(Dropout(0.5))

# Output layer
model.add(Dense(len(word_idx) + 1, activation='softmax'))

# Compile the model
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         1619200   
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                42240     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16192)             1052480   
Total params: 2,718,080
Trainable params: 2,718,080
Non-trainable params: 0
_________________________________________________________________


## Load in Pre-Trained Model

Rather than waiting several hours to train the model, we can load in a model trained for 150 epochs. We'll demonstrate how to train this model for another 5 epochs which shouldn't take too long depending on your hardware.

In [25]:
from keras.models import load_model

# Load in model and demonstrate training
model = load_model('../models/train-embeddings-rnn.h5')
h = model.fit(training_dict['X_train'], training_dict['y_train'], epochs = 5, batch_size = 2048, 
          validation_data = (training_dict['X_valid'], training_dict['y_valid']), 
          verbose = 1)

Train on 222994 samples, validate on 95569 samples
Epoch 1/5
 45056/222994 [=====>........................] - ETA: 7:52 - loss: 3.7202 - acc: 0.2952

KeyboardInterrupt: 

In [26]:
model = load_model('../models/train-embeddings-rnn.h5')
print('Model Performance: Log Loss and Accuracy on training data')
model.evaluate(training_dict['X_train'], training_dict['y_train'], batch_size = 2048)

print('\nModel Performance: Log Loss and Accuracy on validation data')
model.evaluate(training_dict['X_valid'], training_dict['y_valid'], batch_size = 2048)

Model Performance: Log Loss and Accuracy on training data
222994/222994 [==============================] - 182s 816us/step


[3.2820815432408152, 0.33844408377189383]


Model Performance: Log Loss and Accuracy on validation data
95569/95569 [==============================] - 78s 817us/step


[4.737923667784854, 0.2671891513580688]

There is a minor amount of overfitting on the training data but it's not major. Using regularization in both the LSTM layer and after the fully dense layer can help to combat the prevalent issue of overfitting.

# Generate Output

We can use the fully trained model to generate output by starting it off with a seed sequence. The `diversity` controls the amount of stochasticity in the predictions: the next word predicted is selected based on the probabilities of the predictions.

In [13]:
for i in generate_output(model, sequences, idx_word, seed_length = 50, new_words = 30, diversity = 0.75):
    HTML(i)

In [15]:
for i in generate_output(model, sequences, idx_word, seed_length = 30, new_words = 30, diversity = 1.5):
    HTML(i)

Too high of a diversity and the output will be nearly random. Too low of a diversity and the model can get stuck outputting loops of text.

## Start the network with own input

Here you can input your own starting sequence for the network. The network will produce `num_words` of text.

In [16]:
s = 'This patent provides a basis for using a recurrent neural network to '
HTML(seed_sequence(model, s, word_idx, idx_word, diversity = 0.75, num_words = 20))

In [17]:
s = 'The cell state is passed along from one time step to another allowing the '
HTML(seed_sequence(model, s, word_idx, idx_word, diversity = 0.75, num_words = 20))

# Guess if Output is from network or human

The next function plays a simple game: is the output from a human or the network? Two of the choices are computer generated while the third is the actual ending but the order is randomized. Try to see if you can discern the differences! 

In [27]:
guess_human(model, sequences, idx_word)

Seed Sequence: produce a set of minimum distance vectors between the model object and the target object. A neural network estimates translation, rotation, and scaling adjustments that are to be made to the model object. Pose of the model object is adjusted relative to the target object based


Option 1 < --- > on the contents of the selected time. The classification process is a transformed feature data sets, based on a pointer, the DNN classifier using an image containing a plurality of poselets

Option 2 < --- > on the output voltage through the fuzzy rules. The state reference MPC(HM) is input by the normalized channel. An accelerator is computed. An input signal are a detecting unit using a

Option 3 < --- > upon the estimated translation, rotation, and scaling adjustments provided by the neural network. Iterative calculation of the minimum distance vectors, estimation of the translation, rotation, and scaling adjustments


Enter option you think is human (1-3): 3


***Correct***



In [28]:
guess_human(model, sequences, idx_word)

Seed Sequence: higher-order bits in a single clock cycle. Values of the lower-order bits are determined by outputs from comparators with reference voltages provided by digital-to-analog converters. The D/A converters receive inputs from the flash converter as well as from those comparators which provide output results for higher-order bits.


Option 1 < --- > This interconnection of D/A converters and comparators thus forms a

Option 2 < --- > Each amplifier receive columns from coupling area between sum from

Option 3 < --- > A reduced fashion up of background frequencies and sends its


Enter option you think is human (1-3): 1


***Correct***

------------------------------------------------------------
Ordering:  ['human', 'computer0', 'computer1']
Diversity 1.19


In [29]:
guess_human(model, sequences, idx_word)

Seed Sequence: used by an evolutionary algorithm to optimize the neural network architecture. The example initialization parameters include an encoding that determines the initial nodal weights used in each architecture at the commencement of the training cycle. By including the initialization parameters within the encoding used by the evolutionary algorithm


Option 1 < --- > and a neural network. The neural network is used to recognize the parameters of

Option 2 < --- >. The neural network can be used to include a neural network. The training

Option 3 < --- >, the initialization parameters that have a positive effect on the performance of the resultant


Enter option you think is human (1-3): 3


***Correct***

------------------------------------------------------------
Ordering:  ['computer1', 'computer0', 'human']
Diversity 0.51


# Inspect Embeddings

As a final piece of model inspection, we can look at the embeddings and find the words closest to a query word in the embedding space. This gives us an idea of what the network has learned.

In [30]:
embeddings = get_embeddings(model)
embeddings.shape

(16192, 100)

Each word in the vocabulary is now represented as a 100-dimensional vector. This could be reduced to 2 or 3 dimensions for visualization. It can also be used to find the closest word to a query word.

In [31]:
find_closest('network', embeddings, word_idx, idx_word)

Query: network

Word: network         Cosine Similarity: 1.0
Word: channel         Cosine Similarity: 0.7754999995231628
Word: networks        Cosine Similarity: 0.7745000123977661
Word: system          Cosine Similarity: 0.7559999823570251
Word: program         Cosine Similarity: 0.7541999816894531
Word: cable           Cosine Similarity: 0.7419999837875366
Word: now             Cosine Similarity: 0.7297999858856201
Word: programming     Cosine Similarity: 0.7179999947547913
Word: web             Cosine Similarity: 0.7138000130653381
Word: line            Cosine Similarity: 0.6915000081062317


A word should have a cosine similarity of 1.0 with itself! The embeddings are learned for a task, so the nearest words may only make sense in the context of the patents on which we trained the network.

In [32]:
find_closest('data', embeddings, word_idx, idx_word)

Query: data

Word: data            Cosine Similarity: 1.0
Word: information     Cosine Similarity: 0.8185999989509583
Word: numbers         Cosine Similarity: 0.683899998664856
Word: database        Cosine Similarity: 0.6776000261306763
Word: account         Cosine Similarity: 0.6575999855995178
Word: report          Cosine Similarity: 0.6575999855995178
Word: signals         Cosine Similarity: 0.6399999856948853
Word: system          Cosine Similarity: 0.6377000212669373
Word: statistics      Cosine Similarity: 0.6371999979019165
Word: web             Cosine Similarity: 0.6359000205993652


In [46]:
find_closest('embedding', embeddings, word_idx, idx_word)

Query: embedding

Word: embedding       Cosine Similarity: 1.0
Word: Unprocessed     Cosine Similarity: 0.6225000023841858
Word: Summary         Cosine Similarity: 0.5730999708175659
Word: low-dimensional Cosine Similarity: 0.5577999949455261
Word: n-dimensional   Cosine Similarity: 0.5533000230789185
Word: co-occurrence   Cosine Similarity: 0.5504999756813049
Word: articulartory   Cosine Similarity: 0.548799991607666
Word: word-string     Cosine Similarity: 0.5309000015258789
Word: predictional    Cosine Similarity: 0.5260000228881836
Word: input/result    Cosine Similarity: 0.5202999711036682


It seems the network has learned some basic relationships between words! 

# Conclusions

In this notebook, we saw a rapid introduction to recurrent neural networks. The full details can be found in `Deep Dive into Recurrent Neural Networks`. Recurrent neural networks are a powerful tool for natural language processing because of their ability to keep in mind an entire input sequence as they process one word at a time. This makes them applicable to sequence learning tasks where the order of the inputs matter and there can be long-term dependencies in the input sequences. 